## Top 10 arrival airports in the world in 2013 (using the bookings file)

Arrival airport is the column arr_port. It is the IATA code for the airport

To get the total number of passengers for an airport, you can sum the column pax, grouping by arr_port.

Note that there is negative pax. That corresponds to cancelations. So to get the total number of passengers that have actually booked, you should sum including the negatives (that will remove the canceled bookings).

Print the top 10 arrival airports in the standard output, including the number of passengers.

Bonus point: Get the name of the city or airport corresponding to that airport (programatically, we suggest to have a look at GeoBases in Github)

Bonus point: Solve this problem using pandas (instead of any other approach)


In [1]:
!pwd

/home/dsc/Python/DS_Master_A1_Chall/master4


In [3]:
cd ~/Data/challenge/

/home/dsc/Data/challenge


In [4]:
!ls -l

total 1020532
-rw-rw-r--. 1 dsc dsc 554970628 Jan  4  2016 bookings.csv.bz2
-rw-rw-r--  1 dsc dsc    535893 May 19 17:15 bookings.sample.csv.bz2
-rw-rw-r--  1 dsc dsc   4232732 May 19 17:39 bookings.sample.csv.csv
-rw-rw-r--. 1 dsc dsc 483188920 Jan  4  2016 searches.csv.bz2
-rw-rw-r--  1 dsc dsc    244720 May 19 17:20 searches.sample.csv.bz2
-rw-rw-r--  1 dsc dsc   1824927 May 19 17:39 searches.sample.csv.csv


## 1) Get familiar with data

### What if we dont want to read the whole file?

In [6]:
import pandas as pd

In [22]:
#DON'T DO THIS!!!
b = pd.read_csv('bookings.csv.bz2',sep='^')

In [ ]:
b = pd.read_csv('bookings.csv.bz2',sep='^', nrows=1000)

In [7]:
b = pd.read_csv('bookings.sample.csv.bz2',sep='^', nrows=1000)

## Commands for obtaining data info:
   

Commands for obtaining data info:
    b.shape  - object 
    b.info()

    b.columns
    b.columns.to_list()
    list(b.columns)
    
    b.head() -function
    b.sample()
    
    #pd.set_option('display.max_columns', 100)
    pd.set_option('display.max_columns', None)
    
    b.describe()
    b.describe(include='all')
    df.dtypes
    
    b.isnull().sum()
    non_null_counts = b.count()
    

In [9]:
b.shape

(1000, 38)

In [10]:
b.columns

Index([u'act_date           ', u'source', u'pos_ctry', u'pos_iata',
       u'pos_oid  ', u'rloc          ', u'cre_date           ', u'duration',
       u'distance', u'dep_port', u'dep_city', u'dep_ctry', u'arr_port',
       u'arr_city', u'arr_ctry', u'lst_port', u'lst_city', u'lst_ctry',
       u'brd_port', u'brd_city', u'brd_ctry', u'off_port', u'off_city',
       u'off_ctry', u'mkt_port', u'mkt_city', u'mkt_ctry', u'intl',
       u'route          ', u'carrier', u'bkg_class', u'cab_class',
       u'brd_time           ', u'off_time           ', u'pax', u'year',
       u'month', u'oid      '],
      dtype='object')

In [15]:
b.columns.tolist()

['act_date           ',
 'source',
 'pos_ctry',
 'pos_iata',
 'pos_oid  ',
 'rloc          ',
 'cre_date           ',
 'duration',
 'distance',
 'dep_port',
 'dep_city',
 'dep_ctry',
 'arr_port',
 'arr_city',
 'arr_ctry',
 'lst_port',
 'lst_city',
 'lst_ctry',
 'brd_port',
 'brd_city',
 'brd_ctry',
 'off_port',
 'off_city',
 'off_ctry',
 'mkt_port',
 'mkt_city',
 'mkt_ctry',
 'intl',
 'route          ',
 'carrier',
 'bkg_class',
 'cab_class',
 'brd_time           ',
 'off_time           ',
 'pax',
 'year',
 'month',
 'oid      ']

In [16]:
b.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,...,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


In [20]:
#pd.set_option('display.max_columns',100)
pd.set_option('display.max_columns',None)


In [21]:
b.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHRZRH,LONZRH,CHGB,1,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,SAL,SAL,SV,CLT,CLT,US,CLTSAL,CLTSAL,SVUS,1,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLTSAL,CLTSAL,SVUS,1,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKL,AKL,NZ,SVO,MOW,RU,AKLSVO,AKLMOW,NZRU,1,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKLSVO,AKLMOW,NZRU,1,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


In [23]:
b.dtypes

act_date               object
source                 object
pos_ctry               object
pos_iata               object
pos_oid                object
rloc                   object
cre_date               object
duration                int64
distance                int64
dep_port               object
dep_city               object
dep_ctry               object
arr_port               object
arr_city               object
arr_ctry               object
lst_port               object
lst_city               object
lst_ctry               object
brd_port               object
brd_city               object
brd_ctry               object
off_port               object
off_city               object
off_ctry               object
mkt_port               object
mkt_city               object
mkt_ctry               object
intl                    int64
route                  object
carrier                object
bkg_class              object
cab_class              object
brd_time               object
off_time  

In [31]:
b.isnull().sum()

arr_port    0
pax         0
year        0
dtype: int64

In [26]:
b.describe()

,duration,distance,intl,pax,year,month
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.0,1000.0
mean,23935.433000,407.569000,0.643000,0.532000,2013.0,3.0
std,54459.101785,1490.087967,0.479355,1.976077,0.0,0.0
min,61.000000,0.000000,0.000000,-8.000000,2013.0,3.0
25%,3493.000000,0.000000,0.000000,-1.000000,2013.0,3.0
50%,6891.000000,0.000000,1.000000,1.000000,2013.0,3.0
75%,17940.000000,0.000000,1.000000,1.000000,2013.0,3.0
max,482174.000000,10995.000000,1.000000,21.000000,2013.0,3.0


## 2) Select the columns of interest 

In [27]:
b = pd.read_csv('bookings.sample.csv.bz2',sep='^', usecols=['arr_port','pax', 'year'], nrows=1000)

In [28]:
b.head()

,arr_port,pax,year
0,LHR,-1,2013
1,CLT,1,2013
2,CLT,1,2013
3,SVO,1,2013
4,SVO,1,2013


## 3) What to do with NaN?



In [29]:
b=b.dropna()

## Ready to code

What do we have so far?

In [37]:
b = pd.read_csv('bookings.sample.csv.bz2',sep='^', usecols=['arr_port','pax', 'year'], nrows=1000)
b=b.dropna()

## 4) make action plan


#### 4.1) Get only the booking from 2013

In [38]:
b=b[b['year']==2013]

In [39]:
b.drop('year',axis=1, inplace=True)

In [40]:
b.head()

,arr_port,pax
0,LHR,-1
1,CLT,1
2,CLT,1
3,SVO,1
4,SVO,1


#### 4.2) group by arr_port

In [41]:
arr_ports=b.groupby('arr_port')

In [42]:
arr_ports.groups

{'ACC     ': Int64Index([118, 119], dtype='int64'),
 'ADD     ': Int64Index([207, 208, 301, 302], dtype='int64'),
 'AGP     ': Int64Index([397, 398], dtype='int64'),
 'ALA     ': Int64Index([76, 77], dtype='int64'),
 'ALC     ': Int64Index([909, 910], dtype='int64'),
 'ALG     ': Int64Index([34, 35, 36, 37, 558, 559], dtype='int64'),
 'AMD     ': Int64Index([732], dtype='int64'),
 'AMS     ': Int64Index([494, 888, 889, 890, 891], dtype='int64'),
 'ANC     ': Int64Index([432, 433], dtype='int64'),
 'ASU     ': Int64Index([548, 549, 550, 551, 724, 725], dtype='int64'),
 'ATH     ': Int64Index([313, 314, 739, 748], dtype='int64'),
 'ATL     ': Int64Index([147, 404, 908], dtype='int64'),
 'ATQ     ': Int64Index([801, 802, 803], dtype='int64'),
 'AUH     ': Int64Index([733, 734], dtype='int64'),
 'AUS     ': Int64Index([380, 381], dtype='int64'),
 'AYT     ': Int64Index([81, 82, 83, 500], dtype='int64'),
 'BBI     ': Int64Index([151, 152], dtype='int64'),
 'BCN     ': Int64Index([320, 321, 

In [43]:
arr_ports.get_group('ACC     ')

,arr_port,pax
118,ACC,3
119,ACC,3


In [46]:
type(arr_ports)

pandas.core.groupby.DataFrameGroupBy

In [45]:
for airport, val in arr_ports:
    print "%s" %(airport)
    print val['pax'].sum()

ACC     
6
ADD     
2
AGP     
6
ALA     
2
ALC     
4
ALG     
0
AMD     
1
AMS     
1
ANC     
0
ASU     
-10
ATH     
4
ATL     
3
ATQ     
1
AUH     
0
AUS     
2
AYT     
0
BBI     
0
BCN     
5
BEY     
4
BHM     
-1
BJV     
8
BJX     
2
BKK     
2
BLQ     
2
BLR     
0
BNA     
4
BNE     
6
BOG     
0
BOI     
3
BOM     
5
BOS     
11
BRD     
2
BRU     
2
BUD     
0
BUF     
2
BWI     
10
CAG     
0
CAI     
2
CAN     
0
CBR     
0
CCJ     
-3
CCS     
0
CDG     
4
CGK     
12
CHQ     
0
CLT     
5
CMB     
0
COK     
7
COS     
6
CTA     
0
CUN     
4
CUR     
0
CUZ     
0
CVG     
2
CZM     
4
DAB     
2
DAC     
4
DAY     
2
DCA     
50
DEL     
7
DEN     
6
DFW     
-2
DMM     
2
DOH     
3
DTW     
3
DXB     
0
ELP     
0
EWR     
8
EZE     
2
FAI     
4
FAO     
2
FCO     
3
FLL     
13
FRA     
4
FUE     
0
GAU     
-3
GIZ     
0
GRU     
0
GUM     
2
HAV     
0
HER     
4
HKG     
2
HMO     
1
HNL     
0
ICN     
0
IND     
3
ISB     
0
IST     
7
ISU     
0
IXE     
0

#### 4.3) Sum, sort and get top 10

In [47]:
arr_ports.sum()

,pax
arr_port,
ACC,6
ADD,2
AGP,6
ALA,2
ALC,4
ALG,0
AMD,1
AMS,1
ANC,0


In [48]:
arr_ports.sum().sort_values('pax',ascending=False).head(10)

,pax
arr_port,
DCA,50
JFK,14
FLL,13
PVG,13
CGK,12
LGA,12
SVO,12
LAS,11
BOS,11


## 5) Adjust the code to work with Big data

In [49]:
b = pd.read_csv('bookings.sample.csv.bz2',sep='^', usecols=['arr_port','pax', 'year'], nrows=1000)
b=b.dropna()
b=b[b['year']==2013]
b.drop('year',axis=1, inplace=True)
arr_ports=b.groupby('arr_port')
arr_ports.sum().sort_values('pax',ascending=False).head(10)

,pax
arr_port,
DCA,50
JFK,14
FLL,13
PVG,13
CGK,12
LGA,12
SVO,12
LAS,11
BOS,11


In [62]:
b = pd.read_csv('bookings.sample.csv.bz2',sep='^', usecols=['arr_port','pax', 'year']\
                , iterator=True)

In [51]:
type(b)

pandas.io.parsers.TextFileReader

In [52]:
b2 = pd.read_csv('bookings.sample.csv.bz2',sep='^', usecols=['arr_port','pax', 'year'], nrows=1000)
type(b2)

pandas.core.frame.DataFrame

In [63]:
b0=b.get_chunk(1000)

In [54]:
type(b0)

pandas.core.frame.DataFrame

In [64]:
b1=b.get_chunk(8000)

In [65]:
b2=b.get_chunk(3000)

In [66]:
len(b2)

999

In [73]:
b = pd.read_csv('bookings.sample.csv.bz2',sep='^', usecols=['arr_port','pax', 'year']\
                , chunksize=4000)

In [68]:
type(b)

pandas.io.parsers.TextFileReader

In [74]:
b0=b.next()

In [75]:
b1=b.next()

In [76]:
b2=b.get_chunk(4000)

In [77]:
len(b2)

1999

In [79]:
b = pd.read_csv('bookings.sample.csv.bz2',sep='^', usecols=['arr_port','pax', 'year']\
                , chunksize=4000)
for i, chunk in enumerate(b):
    print chunk.head()
    if i==3:
        break

   arr_port  pax  year
0  LHR        -1  2013
1  CLT         1  2013
2  CLT         1  2013
3  SVO         1  2013
4  SVO         1  2013
      arr_port  pax  year
4000  IST         1  2013
4001  IST         1  2013
4002  BWI        -1  2013
4003  BWI         1  2013
4004  BWI        -1  2013
      arr_port  pax  year
8000  LAX         2  2013
8001  LAX        -2  2013
8002  LAX        -2  2013
8003  LAX         2  2013
8004  LAX         2  2013


Here comes the code...

In [82]:
bc = pd.read_csv('bookings.sample.csv.bz2',sep='^', usecols=['arr_port','pax', 'year']\
                , chunksize=4000)
sum_chunks=pd.DataFrame()

for i, b in enumerate(bc):
    print "Chunk: %d, size of chunk %d"%(i+1, len(b))
    b=b.dropna()
    b=b[b['year']==2013]
    b.drop('year',axis=1, inplace=True)
    arr_ports=b.groupby('arr_port')
    chunk_result= arr_ports.sum()
    sum_chunks=sum_chunks.append(chunk_result)
    
    #.sort_values('pax',ascending=False)
    #.head(10)
sum_chunks.head()    
    
    

Chunk: 1, size of chunk 4000
Chunk: 2, size of chunk 4000
Chunk: 3, size of chunk 1999


,pax
arr_port,
ABQ,4
ABV,0
ABZ,-2
ACC,0
ADD,4


In [83]:
sum_chunks.head()



,pax
arr_port,
ABQ,4
ABV,0
ABZ,-2
ACC,0
ADD,4


We have to groupby, sum and sort all the results which correspond to the processing of a single chunk 

In [88]:
Top_airports=sum_chunks.groupby(sum_chunks.index)\
    .pax.sum().sort_values(ascending=False).reset_index()

In [89]:
Top_airports.head()

,arr_port,pax
0,HKG,112
1,LGA,95
2,ORD,94
3,JFK,92
4,SFO,91


Compare the results to the simple approach without the chunks. This is to be sure that the concatenation of the chunk results is ok.

In [91]:
b = pd.read_csv('bookings.sample.csv.bz2',sep='^', usecols=['arr_port','pax', 'year'])
b=b.dropna()
b=b[b['year']==2013]
b.drop('year',axis=1, inplace=True)
arr_ports=b.groupby('arr_port')
arr_ports.sum().sort_values('pax',ascending=False).head()

,pax
arr_port,
HKG,112
LGA,95
ORD,94
JFK,92
LAX,91


Ready to process the BIG file!!!

In [93]:
%%time
bc = pd.read_csv('bookings.csv.bz2',sep='^', usecols=['arr_port','pax', 'year']\
                , chunksize=1000000)
sum_chunks=pd.DataFrame()

for i, b in enumerate(bc):
    print "Chunk: %d, size of chunk %d"%(i+1, len(b))
    b=b.dropna()
    b=b[b['year']==2013]
    b.drop('year',axis=1, inplace=True)
    arr_ports=b.groupby('arr_port')
    chunk_result= arr_ports.sum()
    sum_chunks=sum_chunks.append(chunk_result)

Top_airports=sum_chunks.groupby(sum_chunks.index)\
    .pax.sum().sort_values(ascending=False).reset_index()    

Chunk: 1, size of chunk 1000000
Chunk: 2, size of chunk 1000000
Chunk: 3, size of chunk 1000000
Chunk: 4, size of chunk 1000000
Chunk: 5, size of chunk 1000000
Chunk: 6, size of chunk 1000000
Chunk: 7, size of chunk 1000000
Chunk: 8, size of chunk 1000000
Chunk: 9, size of chunk 1000000
Chunk: 10, size of chunk 1000000
Chunk: 11, size of chunk 10
CPU times: user 4min 30s, sys: 1.11 s, total: 4min 31s
Wall time: 4min 36s


In [94]:
Top_airports.head(10)

,arr_port,pax
0,LHR,88809.0
1,MCO,70930.0
2,LAX,70530.0
3,LAS,69630.0
4,JFK,66270.0
5,CDG,64490.0
6,BKK,59460.0
7,MIA,58150.0
8,SFO,58000.0
9,DXB,55590.0


In [95]:
len(Top_airports)

2274

In [ ]:
#install it with: pip install GeoBases

In [96]:
from GeoBases import GeoBase

In [99]:
geoDict=GeoBase(data='ori_por',verbose=False)

In [101]:
geoDict.keys()[:10]

['MLC',
 'XMZ',
 'XQP@1',
 'JED@1',
 'MFI@1',
 'JGO@1',
 'YAL@1',
 'NDP',
 'GEG@1',
 'NDS']

In [102]:
geoDict.fields

['__key__',
 '__dup__',
 '__par__',
 '__lno__',
 'iata_code',
 'icao_code',
 'faa_code',
 'is_geonames',
 'geoname_id',
 'valid_id',
 'name',
 'asciiname',
 'lat',
 'lng',
 'fclass',
 'fcode',
 'page_rank',
 'date_from',
 'date_until',
 'comment',
 'country_code',
 'cc2',
 'country_name',
 'continent_name',
 'adm1_code',
 'adm1_name_utf',
 'adm1_name_ascii',
 'adm2_code',
 'adm2_name_utf',
 'adm2_name_ascii',
 'adm3_code',
 'adm4_code',
 'population',
 'elevation',
 'gtopo30',
 'timezone',
 'gmt_offset',
 'dst_offset',
 'raw_offset',
 'moddate',
 'city_code',
 'city_name_utf',
 'city_name_ascii',
 'tvl_por_list@raw',
 'tvl_por_list',
 'state_code',
 'location_type@raw',
 'location_type',
 'wiki_link',
 'alt_name_section@raw',
 'alt_name_section',
 '__gar__']

In [103]:
geoDict.get?

In [104]:
geoDict.get('LHR', 'city_name_utf')

'London'

In [105]:
geoDict.get('LHR')['name']

'London Heathrow Airport'

In [107]:
Top_airports['arr_port'][0]

'LHR     '

In [108]:
Top_airports['arr_port'][0].strip()

'LHR'

In [109]:
geoDict.get(Top_airports['arr_port'][0].strip())['name']

'London Heathrow Airport'

In [113]:
Top_airports['Airport Name']=Top_airports['arr_port'].\
map(lambda x: geoDict.get(x.strip())['name'])

In [114]:
Top_airports.head()

,arr_port,pax,Airport Name
0,LHR,88809.0,London Heathrow Airport
1,MCO,70930.0,Orlando International Airport
2,LAX,70530.0,Los Angeles International Airport
3,LAS,69630.0,Mc Carran International Airport
4,JFK,66270.0,John F Kennedy International Airport


Save to file so we can use it in exercise 5.

In [115]:
Top_airports.to_csv('top_airports.csv', sep='^', index=False)

In [116]:
!ls -l

total 1020616
-rw-rw-r--. 1 dsc dsc 554970628 Jan  4  2016 bookings.csv.bz2
-rw-rw-r--  1 dsc dsc    535893 May 19 17:15 bookings.sample.csv.bz2
-rw-rw-r--  1 dsc dsc   4232732 May 19 17:39 bookings.sample.csv.csv
-rw-rw-r--. 1 dsc dsc 483188920 Jan  4  2016 searches.csv.bz2
-rw-rw-r--  1 dsc dsc    244720 May 19 17:20 searches.sample.csv.bz2
-rw-rw-r--  1 dsc dsc   1824927 May 19 17:39 searches.sample.csv.csv
-rw-rw-r--  1 dsc dsc     84696 May 19 20:55 top_airports.csv
